<a href="https://colab.research.google.com/github/existinmyhead/Winter22/blob/main/CIS3200/Project5/Assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pyspark

!wget -q 'https://drive.google.com/uc?export=download&id=1wAdLRbT_YgWJ1XRl39JM-lKwJpewdwyM' -O 'Sample.txt'

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 39.1 MB/s 


In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

conf = SparkConf().setAppName('SparkWordCount')
sc = SparkContext.getOrCreate(conf = conf)

sqlContext = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
input_file = sc.textFile('Friends.txt')

# input_file.collect()[:10]

In [ ]:
# Split lines by whitespace and remove all commas from text

import re
# flatMapVar = input_file.map(lambda line: re.split(r"[ ,\t]+", line))
flatMapVar = input_file.map(lambda line: re.findall(r"[0-9]+", line))

# flatMapVar.collect()[:10]

In [ ]:
# Remove users with no friends

flatMapFriend = flatMapVar.filter(lambda line: len(line) > 1)

# flatMapFriend.collect()[:10]

In [ ]:
# Expand list into key-value pairs
# Creates list of pairs of actual friends

def f(x): return x
flatMapFriendPair = flatMapFriend.map(lambda line: (line[0], line[1:])).flatMapValues(f).collect()

# flatMapFriendPair

In [ ]:
# Create data dictionary to iterate through for middle friend list

dictionary = {}
for line in flatMapFriend.collect():
  dictionary[line[0]] = line[1:]

# dictionary

In [ ]:
# Creates list of all pairs with a friend in the middle

# USING ACTUAL FRIEND PAIRS LIST

# create empty middle friend list
# for loop to iterate through actual friend pairs list, line by line
# save first element of tuple as base friend
# save second element of tuple as user friend
  # for loop to iterate through all tuples with first element being user friend
  # extract second element of tuple and pair with base friend iff it is not equal to base friend
  # add created tuple to middle friends list

friendMiddleList = []

for pair in flatMapFriendPair:
  
  baseFriend = pair[0]
  userFriend = pair[1]
  fFriend = dictionary[userFriend][:]
  
  fFriend.remove(baseFriend)
  mutualFriend = [(baseFriend, recFriend) for recFriend in fFriend]
  friendMiddleList.extend(mutualFriend)

# flatMapMiddlePair = sc.parallelize(friendMiddleList)
# flatMapMiddlePair.collect()[:15]

In [ ]:
# Creates list of mutual friend pairs
# MIDDLE FRIEND PAIRS LIST - ACTUAL FRIEND PAIRS LIST

flatMapMutualPair = sc.parallelize(list(set(friendMiddleList) - set(flatMapFriendPair)))
# flatMapMutualPair.collect()[0]

In [ ]:
# To rank mutual pairs by count:
# go through each tuple and assign count = 1
# reduce and combine the tuples that are repeats so count = sum of those counts
# re-order the list based on the counts (higher count = more mutual friends)
# limit the new list to 10 recs based on the order from above code

flatMapRankedPair = flatMapMutualPair.map(lambda x: (x, 1))
countKeys = flatMapRankedPair.reduceByKey(lambda a, b: a + b)
flatMapMutualPair = countKeys.sortBy(lambda x: -x[1]).keys().groupByKey().mapValues(list).map(lambda x: (x[0], x[1][:10]))


In [ ]:
# Add removed users (had no friends, closed circle of friends) back to final list

original = flatMapVar.map(lambda line: (line[0], line[1:])).keys()

# original.collect()

trimmed = flatMapMutualPair.keys()

# trimmed.collect()

removed = original.subtract(trimmed).map(lambda line: (line, ""))

# removed.collect()

trueMutuals = flatMapMutualPair.union(removed)

# trueMutuals.collect()[:15]

In [ ]:
recs = sqlContext.createDataFrame(trueMutuals) \
            .withColumnRenamed('_1','User').withColumnRenamed('_2','Recommendations')

recs.toPandas().to_csv('friend_recs.csv', index=False)

# recs.show(truncate = False)

Here's all the above steps merged into one block of code:

In [ ]:
# Import input file
input_file = sc.textFile('Friends.txt')


# Split lines by whitespace and remove all commas from text
import re

flatMapVar = input_file.map(lambda line: re.findall(r"[0-9]+", line))


# Remove users with no friends
flatMapFriend = flatMapVar.filter(lambda line: len(line) > 1)


# Create list of pairs of actual friends
def f(x): return x

flatMapFriendPair = flatMapFriend.map(lambda line: (line[0], line[1:])).flatMapValues(f).collect()


# Create data dictionary to iterate through for middle friend list
dictionary = {}

for line in flatMapFriend.collect():
  dictionary[line[0]] = line[1:]


# Create list of all pairs with a friend in the middle
friendMiddleList = []

for pair in flatMapFriendPair:
  
  baseFriend = pair[0]
  userFriend = pair[1]
  fFriend = dictionary[userFriend][:]
  
  fFriend.remove(baseFriend)
  mutualFriend = [(baseFriend, recFriend) for recFriend in fFriend]
  friendMiddleList.extend(mutualFriend)


# Creates list of mutual friend pairs
flatMapMutualPair = sc.parallelize(list(set(friendMiddleList) - set(flatMapFriendPair)))


# Rank mutual pairs list by count and limit recs to 10
flatMapRankedPair = flatMapMutualPair.map(lambda x: (x, 1))
countKeys = flatMapRankedPair.reduceByKey(lambda a, b: a + b)
flatMapMutualPair = countKeys.sortBy(lambda x: -x[1]).keys().groupByKey().mapValues(list).map(lambda x: (x[0], x[1][:10]))


# Add removed users (had no friends, closed circle of friends) back to final list
original = flatMapVar.map(lambda line: (line[0], line[1:])).keys()
trimmed = flatMapMutualPair.keys()
removed = original.subtract(trimmed).map(lambda line: (line, ""))

trueMutuals = flatMapMutualPair.union(removed)


# Create dataframe and export csv
recs = sqlContext.createDataFrame(trueMutuals) \
            .withColumnRenamed('_1','User').withColumnRenamed('_2','Recommendations')

recs.toPandas().to_csv('friend_recs.csv', index=False)